In [19]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict

In [3]:
collection_name = 'week4_residual_freeze_projection'
collection = seml.database.get_collection(collection_name)
results = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]

In [6]:
def flatten_dict(d):
    new = dict()
    for k, v in d.items():
        if isinstance(v, dict):
            for kk, vv in flatten_dict(v).items():
                new[k + '.' + kk] = vv
        else:
            new[k] = v
    return new
    

{'config': {'overwrite': 1,
  'db_collection': 'week4_residual_freeze_projection',
  'data': {'dataset': 'cora_ml',
   'num_dataset_splits': 5,
   'split_type': 'uniform',
   'test_portion': 0.6,
   'test_portion_fixed': 0.2,
   'train_labels': 'all',
   'train_portion': 20,
   'val_labels': 'all',
   'val_portion': 20},
  'evaluation': {'pipeline': []},
  'model': {'activation': 'leaky_relu',
   'freeze_residual_projection': True,
   'hidden_sizes': [64],
   'leaky_relu_slope': 0.01,
   'model_type': 'gcn',
   'num_initializations': 5,
   'residual': True,
   'use_bias': True,
   'use_spectral_norm': True,
   'weight_scale': 0.5},
  'run': {'args': ['model.weight_scale',
    'model.residual',
    'model.hidden_sizes',
    'model.freeze_residual_projection'],
   'name': 'gcn_{2}_ws_{0}_res_{1}_frozen_{3}'},
  'training': {'early_stopping': {'min_delta': 0.001,
    'mode': 'min',
    'monitor': 'val_loss',
    'patience': 100},
   'gpus': 1,
   'learning_rate': 0.001,
   'max_epochs': 1

In [20]:
# Collect everything into a data frame
df = defaultdict(list)
for result in results:
    for k, v in flatten_dict(result['config']).items():
        df[k].append(v)
    with open(result['result']) as f:
        metrics = json.load(f)
        for metric, values in metrics.items():
            df[metric + '.mean'].append(np.array(values).mean())
            df[metric + '.std'].append(np.array(values).std())
df = pd.DataFrame(df)
    

In [26]:
df.columns

Index(['overwrite', 'db_collection', 'data.dataset', 'data.num_dataset_splits',
       'data.split_type', 'data.test_portion', 'data.test_portion_fixed',
       'data.train_labels', 'data.train_portion', 'data.val_labels',
       'data.val_portion', 'evaluation.pipeline', 'model.activation',
       'model.freeze_residual_projection', 'model.hidden_sizes',
       'model.leaky_relu_slope', 'model.model_type',
       'model.num_initializations', 'model.residual', 'model.use_bias',
       'model.use_spectral_norm', 'model.weight_scale', 'run.args', 'run.name',
       'training.early_stopping.min_delta', 'training.early_stopping.mode',
       'training.early_stopping.monitor', 'training.early_stopping.patience',
       'training.gpus', 'training.learning_rate', 'training.max_epochs',
       'seed', 'val_loss.mean', 'val_loss.std', 'val_accuracy.mean',
       'val_accuracy.std'],
      dtype='object')

In [42]:
sub_df = df[df['model.freeze_residual_projection'] == True].sort_values('val_accuracy.mean', ascending=False)
sub_df[['model.hidden_sizes', 'model.weight_scale', 'val_accuracy.mean', 'val_accuracy.std']]

,model.hidden_sizes,model.weight_scale,val_accuracy.mean,val_accuracy.std
48,"[64, 64]",3.0,0.844286,0.016782
28,"[64, 32]",2.0,0.840000,0.022223
46,"[64, 64]",2.0,0.839143,0.021582
30,"[64, 32]",3.0,0.838286,0.023809
50,"[64, 64]",4.0,0.833143,0.019047
32,"[64, 32]",4.0,0.832571,0.022685
14,[64],4.0,0.829429,0.024438
26,"[64, 32]",1.5,0.828571,0.018183
44,"[64, 64]",1.5,0.826857,0.025736
16,[64],5.0,0.824286,0.021946


In [43]:
sub_df = df[(df['model.freeze_residual_projection'] == False) & (df['model.residual'] == True)].sort_values('val_accuracy.mean', ascending=False)
sub_df[['model.hidden_sizes', 'model.weight_scale', 'val_accuracy.mean', 'val_accuracy.std']]

,model.hidden_sizes,model.weight_scale,val_accuracy.mean,val_accuracy.std
45,"[64, 64]",1.5,0.843143,0.025593
27,"[64, 32]",1.5,0.843143,0.022989
47,"[64, 64]",2.0,0.840857,0.025114
51,"[64, 64]",4.0,0.840571,0.014808
13,[64],3.0,0.840571,0.027607
49,"[64, 64]",3.0,0.839429,0.018965
29,"[64, 32]",2.0,0.837714,0.022992
25,"[64, 32]",1.1,0.836286,0.024571
43,"[64, 64]",1.1,0.836000,0.025192
11,[64],2.0,0.835714,0.030102


In [44]:
sub_df = df[(df['model.freeze_residual_projection'] == False) & (df['model.residual'] == False)].sort_values('val_accuracy.mean', ascending=False)
sub_df[['model.hidden_sizes', 'model.weight_scale', 'val_accuracy.mean', 'val_accuracy.std']]

,model.hidden_sizes,model.weight_scale,val_accuracy.mean,val_accuracy.std
78,"[64, 64]",3.0,0.851143,0.021255
69,"[64, 32]",3.0,0.849429,0.016893
62,[64],5.0,0.844286,0.022946
79,"[64, 64]",4.0,0.843143,0.020651
61,[64],4.0,0.842000,0.021507
70,"[64, 32]",4.0,0.841143,0.021216
68,"[64, 32]",2.0,0.840571,0.015081
77,"[64, 64]",2.0,0.838000,0.015215
80,"[64, 64]",5.0,0.836000,0.020251
71,"[64, 32]",5.0,0.836000,0.018896
